In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam


In [2]:
# Membaca file CSV
df = pd.read_csv('data_harga_beras.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date').interpolate(method='linear').reset_index()

In [3]:
# Drop kolom yang tidak digunakan
df = df.drop(['C4Super', 'C4Medium'], axis=1)
df = df.filter(['Bulog'])
df = df.values.astype('float32')

In [4]:
# Normalisasi data dengan range [0, 1] sebelum membagi data
scaler_0_1 = MinMaxScaler(feature_range=(0, 1))
df_normalized_0_1 = scaler_0_1.fit_transform(df)

scaler_neg1_1 = MinMaxScaler(feature_range=(-1, 1))
df_normalized_neg1_1 = scaler_neg1_1.fit_transform(df)

In [5]:
# Fungsi untuk menyiapkan data
def prepare_data(df, timesteps):
    X, Y = [], []
    for i in range(len(df) - timesteps - 1):
        X.append(df[i:(i + timesteps)])
        Y.append(df[i + timesteps, 0])
    return np.array(X), np.array(Y)

In [6]:
# Fungsi untuk membangun dan melatih model
def build_and_train_model(df_normalized, timesteps=1, num_neuron=10, num_epoch=50, num_batch=16):
    # Split data menjadi training dan testing
    n_samples = len(df_normalized)
    train_size = int(n_samples * 0.9)
    test_size = n_samples - train_size

    train_data = df_normalized[:train_size]
    test_data = df_normalized[train_size:]
    
    # Persiapan data
    X_train, Y_train = prepare_data(train_data, timesteps)
    X_test, Y_test = prepare_data(test_data, timesteps)

    # Reshape data
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    # Build model
    model = Sequential()
    model.add(LSTM(4, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(num_neuron, activation='tanh', recurrent_activation='sigmoid', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, bias_initializer='zeros'))
    
    # Compile model
    adam_optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=adam_optimizer, loss='mse', metrics=['MAPE'])
    
    # Train model
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=num_epoch, batch_size=num_batch, verbose=1)
    
    return model, X_test, Y_test, history


In [7]:
# Normalisasi dengan range [0, 1]
model_0_1, X_test_0_1, Y_test_0_1, history_0_1 = build_and_train_model(df_normalized_0_1)

# Normalisasi dengan range [-1, 1]
model_neg1_1, X_test_neg1_1, Y_test_neg1_1, history_neg1_1 = build_and_train_model(df_normalized_neg1_1)

C:\Users\Windows\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - MAPE: 42505.6836 - loss: 0.1040 - val_MAPE: 81.6306 - val_loss: 0.5938
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 411473.8438 - loss: 0.0563 - val_MAPE: 64.3536 - val_loss: 0.3694
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 219031.4219 - loss: 0.0346 - val_MAPE: 55.5147 - val_loss: 0.2750
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 75339.7109 - loss: 0.0323 - val_MAPE: 49.5449 - val_loss: 0.2190
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 417424.7500 - loss: 0.0250 - val_MAPE: 38.3101 - val_loss: 0.1309
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 155364.1719 - loss: 0.0160 - val_MAPE: 25.3287 - val_loss: 0.0573
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 106531.3516 - loss: 0.0129 - val_MAPE: 16.0554 - val_loss: 0.0230
Epoch 8/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 25745.9746 - loss: 0.0075 - val_MAPE: 13.6313 - val_loss: 0.0166
Epoch 9/50

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 35.0265 - loss: 0.1252 - val_MAPE: 110.7906 - val_loss: 0.9638
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 32.2871 - loss: 0.1025 - val_MAPE: 93.5015 - val_loss: 0.6872
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 28.1991 - loss: 0.0698 - val_MAPE: 68.8192 - val_loss: 0.3734
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 25.9866 - loss: 0.0505 - val_MAPE: 45.6124 - val_loss: 0.1651
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 22.0233 - loss: 0.0338 - val_MAPE: 26.9123 - val_loss: 0.0584
Epoch 8/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 21.2595 - loss: 0.0262 - val_MAPE: 15.6386 - val_loss: 0.0205
Epoch 9/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 20.3505 - loss: 0.0259 - val_MAPE: 15.7219 - val_loss: 0.0207
Epoch 10/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 18.5696 - loss: 0.0254 - val_MAPE: 15.7224 - val_loss: 0.0207
Epoch 11/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/ste

In [8]:
# Evaluasi model sebelum denormalisasi
def evaluate_model_before_denormalization(model, X_test, Y_test):
    # Predict
    test_predict = model.predict(X_test, verbose=0)

    # Calculate metrics
    mae = mean_absolute_error(Y_test, test_predict)
    rmse = sqrt(mean_squared_error(Y_test, test_predict))
    mape = mean_absolute_percentage_error(Y_test, test_predict) * 100

    return mae, rmse, mape, Y_test, test_predict

In [9]:

# Evaluasi model untuk kedua skala
mae_0_1, rmse_0_1, mape_0_1, Y_test_0_1, test_predict_0_1 = evaluate_model_before_denormalization(model_0_1, X_test_0_1, Y_test_0_1)
mae_neg1_1, rmse_neg1_1, mape_neg1_1, Y_test_neg1_1, test_predict_neg1_1 = evaluate_model_before_denormalization(model_neg1_1, X_test_neg1_1, Y_test_neg1_1)

# Print hasil evaluasi
print(f"Interval [0, 1] -> RMSE: {rmse_0_1}, MAPE: {mape_0_1}%")
print(f"Interval [-1, 1] -> RMSE: {rmse_neg1_1}, MAPE: {mape_neg1_1}%")


Interval [0, 1] -> RMSE: 0.04670875095450275, MAPE: 4.840087890625%
Interval [-1, 1] -> RMSE: 0.10412913828585423, MAPE: 10.104700922966003%
